In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Im merging data over here but ill not perform any analysis 


all analysis is performed at mergedata.ipynb


In [3]:
df2 = pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\pgen.1006501.s002.xlsx') #s002 to s007 all imported
df3=pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\pgen.1006501.s003.xlsx')
df4 = pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\s004p1.xlsx')
df5=pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\pgen.1006501.s005.xlsx')
df6=pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\pgen.1006501.s006.xlsx')
df7=pd.read_excel(r'C:\Users\Advaitha\Desktop\projects\GENOME\pgen.1006501.s007.xlsx')

c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
merged_df = pd.merge(df2, df5, on='Gene', how='left')

In [6]:
print(merged_df.columns)

Index(['Patient.ID', 'Sample.ID', 'Primary_Relapse', 'Has.Multiple.Samples',
       'Coverage', 'Event.Category', 'Lesion', 'Gene', 'Event',
       'Age.at.Diagnosis', 'Stage', 'COG.Risk.Group', 'MYCN.Gene.Status',
       'DNA.Index', 'Pathology.Grade', 'Therapy.Recieved',
       'HR.induction.chemo', 'HR.consolidation.chemo', 'Radiation.therapy',
       'MIBG.therapy', 'Isotretinoin.therapy', 'Anti.GD2.immunotherapy',
       'IR.chemo', 'Targeted.Treatment.FM', 'Response.FM.Based.Therapy',
       'Duration.Response', 'Variant status', 'Potentially actionable'],
      dtype='object')


In [7]:
merged_df['Gene'] = merged_df['Gene'].str.upper().str.strip()

In [8]:
merged_df['Potentially actionable'].fillna('Unknown', inplace=True)

In [15]:
print(merged_df['DNA.Index'])

0       No Data
1       No Data
2       No Data
3       No Data
4       No Data
         ...   
1602       1.44
1603       1.44
1604       1.44
1605       1.44
1606       1.44
Name: DNA.Index, Length: 1607, dtype: object


In [10]:
print(merged_df.columns)

Index(['Patient.ID', 'Sample.ID', 'Primary_Relapse', 'Has.Multiple.Samples',
       'Coverage', 'Event.Category', 'Lesion', 'Gene', 'Event',
       'Age.at.Diagnosis', 'Stage', 'COG.Risk.Group', 'MYCN.Gene.Status',
       'DNA.Index', 'Pathology.Grade', 'Therapy.Recieved',
       'HR.induction.chemo', 'HR.consolidation.chemo', 'Radiation.therapy',
       'MIBG.therapy', 'Isotretinoin.therapy', 'Anti.GD2.immunotherapy',
       'IR.chemo', 'Targeted.Treatment.FM', 'Response.FM.Based.Therapy',
       'Duration.Response', 'Variant status', 'Potentially actionable'],
      dtype='object')


In [23]:
def convert_to_numeric(df, columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [24]:
columns_to_convert = ['Coverage', 'Age.at.Diagnosis', 'DNA.Index']
merged_df = convert_to_numeric(merged_df, columns_to_convert)

In [27]:
merged_df.replace(['No Data', 'Unknown'], np.nan, inplace=True)

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import shap

ModuleNotFoundError: No module named 'shap'

In [47]:
def extract_numeric(value):
    if isinstance(value, str):
        match = re.search(r'(\d+)', value)
        if match:
            return float(match.group(1))
    return np.nan

# Apply function to columns with mixed types
merged_df['Age.at.Diagnosis'] = merged_df['Age.at.Diagnosis'].apply(extract_numeric)
merged_df['DNA.Index'] = merged_df['DNA.Index'].apply(extract_numeric)

In [49]:
# Convert other columns to numeric where applicable
def convert_to_numeric(df, columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Columns to convert
columns_to_convert = ['Coverage']
merged_df = convert_to_numeric(merged_df, columns_to_convert)

In [51]:
# Replace placeholder strings with NaN
merged_df.replace(['No Data', 'Unknown'], np.nan, inplace=True)

In [52]:
# Impute missing values
imputer_numeric = SimpleImputer(strategy='mean')
merged_df[['Age.at.Diagnosis', 'DNA.Index'] + columns_to_convert] = imputer_numeric.fit_transform(merged_df[['Age.at.Diagnosis', 'DNA.Index'] + columns_to_convert])

c:\Users\Advaitha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['Age.at.Diagnosis' 'DNA.Index']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: Columns must be same length as key

In [41]:
le = LabelEncoder()
for col in categorical_columns:
    merged_df[col] = le.fit_transform(merged_df[col].astype(str))

In [42]:
X = merged_df.drop(columns=['Patient.ID', 'Sample.ID', 'Response.FM.Based.Therapy'])
y = merged_df['Response.FM.Based.Therapy']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

ValueError: could not convert string to float: '12 months'